In [2]:
from pattern.web import URL, DOM, plaintext, Element, extension, Crawler, DEPTH
import re
import pickle
import random
import PIL
import time
from PIL import Image
import os

In [3]:
class Scraper():
    def save_image(self):
        pass
    def get_recipe(self):
        pass

In [4]:
class AllRecipesScraper(Scraper):
    
    def save_image(self,element,idx,basewidth = 300):
        first_rec_photo = element.by_class("rec-photo")[0]
        url = first_rec_photo.attributes.get('src','')
        if '/nophoto/nopicture' not in url:
            img_url = URL(url)
            img = Image.open(img_url)
            wpercent = (basewidth / float(img.size[0]))
            hsize = int((float(img.size[1]) * float(wpercent)))
            img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
            img.save("img/"+str(idx) + extension(img_url.page))
            return True
        else:
            return False
        
        
    def get_ingredients(self,element):
        ing_nodes = element.by_class("recipe-ingred_txt added")
        return "\n".join([plaintext(a.content) for a in ing_nodes])

    
    def get_instructions(self,element):
        instr_nodes = element.by_class("recipe-directions__list--item")
        return "\n".join([plaintext(a.content) for a in instr_nodes])
    
    def get_recipe(self,element):
        return self.get_ingredients(element)+"\n"+self.get_instructions(element)

In [5]:
class AllRecipesRandomSearch():
    def __init__(self,tried_ids = None, recipe_list = None ):
        self.scraper = AllRecipesScraper()
        if tried_ids is None:
            self.tried_ids = set()
        else:
            self.tried_ids = tried_ids
        if recipe_list is None:
            self.recipe_list = {}
        else:
            self.recipe_list = recipe_list
        self.count = 0
        
    def reset_count(self):
        self.count = 0
        
    def new_id(self,rec_id):
        return rec_id not in self.tried_ids
          
    def visit(self,rec_id):
        self.count += 1
        url = URL("http://allrecipes.com/recipe/"+str(rec_id))
        try:
            source = url.download(cached=True)
            self.scrape(source, rec_id)
        except Exception as detail:
            print 'Unable to Scrape:', rec_id
            self.tried_ids.add(rec_id)
            
    def scrape(self,source,rec_id):
        print("scraping", rec_id)
        element = Element(source)
        saved = self.scraper.save_image(element, rec_id)
        if saved:
            recipe = self.scraper.get_recipe(element)
            self.recipe_list[rec_id]=recipe
        else:
            print "Image less recipe "
        self.tried_ids.add(rec_id)
        

In [7]:
tried_ids = pickle.load(open("recipe_lists/tried_ids_AL.p","rb"))
recipe_list = pickle.load(open("recipe_lists/recipe_list_AL.p","rb"))

print len(tried_ids)

search = AllRecipesRandomSearch(tried_ids,recipe_list)
limit =10000
rec_id = 249249+40000

while search.count < limit:
    if search.count%100 ==0:
        print "Check point Iter: %s"%search.count
        pickle.dump( search.recipe_list, open( "recipe_lists/recipes"+str(search.count/500)+".p", "wb" ) )
        pickle.dump( search.tried_ids, open( "recipe_lists/tried_ids"+str(search.count/500)+".p", "wb" ) )
    if search.new_id(rec_id):
        search.visit(rec_id)
    rec_id+=1
    time.sleep(1)

print("FINAL REC ID",rec_id)
print "Saving the final work"
pickle.dump( search.recipe_list, open( "recipe_lists/recipes_final_AL10.p", "wb" ) )
pickle.dump( search.tried_ids, open( "recipe_lists/tried_ids_final_AL10.p", "wb" ) )

3759
Check point Iter: 0
Unable to Scrape: 289249
Unable to Scrape: 289250
Unable to Scrape: 289251
Unable to Scrape: 289252
Unable to Scrape: 289253
Unable to Scrape: 289254
Unable to Scrape: 289255
Unable to Scrape: 289256
Unable to Scrape: 289257
Unable to Scrape: 289258
Unable to Scrape: 289259
Unable to Scrape: 289260
Unable to Scrape: 289261
Unable to Scrape: 289262
Unable to Scrape: 289263
Unable to Scrape: 289264
Unable to Scrape: 289265
Unable to Scrape: 289266
Unable to Scrape: 289267
Unable to Scrape: 289268
Unable to Scrape: 289269
Unable to Scrape: 289270
Unable to Scrape: 289271
Unable to Scrape: 289272
Unable to Scrape: 289273
Unable to Scrape: 289274
Unable to Scrape: 289275
Unable to Scrape: 289276
Unable to Scrape: 289277
Unable to Scrape: 289278
Unable to Scrape: 289279
Unable to Scrape: 289280
Unable to Scrape: 289281
Unable to Scrape: 289282
Unable to Scrape: 289283
Unable to Scrape: 289284
Unable to Scrape: 289285
Unable to Scrape: 289286
Unable to Scrape: 289287
